In [1]:
import torch
import torch.nn as nn 
import numpy as np 
import pandas as pd 
import torch.nn.functional as F

In [2]:
import IPython
from PIL import Image
import matplotlib.pyplot as plt

In [3]:
df=pd.read_excel('./DS_Assignment_2/height_and_pose.xlsx')

In [4]:
import json

In [5]:
df.head()

,Depthmap Image,Height(cm),Pose
0,1d5fee00-ada4-11eb-a80b-3f50af21830f,76.7,"[{'bbox_coordinates': [(158.91087, 92.53168), ..."
1,1eddf4c0-ad9f-11eb-a80b-ef4adba049aa,66.4,"[{'bbox_coordinates': [(131.00702, 68.115364),..."
2,38e29bc0-8fc0-11eb-b5f2-0742da91b282,63.2,"[{'bbox_coordinates': [(116.070114, 37.567608)..."
3,6cd5d480-8b8b-11eb-b152-f7b115384fa9,87.1,"[{'bbox_coordinates': [(145.24242, 26.49348), ..."
4,76ed6fc0-8e1d-11eb-ad70-7be1a1219834,59.1,"[{'bbox_coordinates': [(150.17046, 65.00379), ..."


In [6]:
import cv2

In [7]:
def changeshape(layer):
    x = F.relu(F.max_pool2d(F.conv2d(layer), kernel_size=2, stride=2))  # reduce spatial dimensions to (112, 112)
    x = F.relu(F.max_pool2d(F.conv2d(layer), kernel_size=2, stride=2)) 
    downsampled_tensor = x
    resized_layer = F.interpolate(downsampled_tensor, size=(54,54,1), mode='bilinear')
    return resized_layer


In [94]:
def imageread(path,pose):
    # img=Image.open(path)
    # img = img.resize((224, 224), resample=Image.BILINEAR)
    img=cv2.imread(path)
    img=cv2.resize(img,(224,224))
    st='bbox_confidence_score'
    layer=torch.zeros_like(torch.tensor(img[:,:,0]))
    pose=pose[1:-1]
    pose=pose.replace('\'','\"')
    pose=pose.replace(" ","")
    p=0
    while(st!=pose[p:p+21]):
        p=p+1
    pose='{'+pose[p-1:]
    # print(pose[0])
    pose=json.loads(pose)
    for i1,i2 in zip(pose['key_points_coordinate'],pose['key_points_prob']):
        for j1,j2 in zip(i1,i2):
            x=int((i1[j1]['x']/480)*224)
            y=int((i1[j1]['y']/640)*224)
            prob=i2[j2]['score']
            # print(prob)
            layer[x][y]=int((1-prob)*255.0)
    return img,layer     

In [9]:
def createModel(printtoggle=False):

  class model(nn.Module):
    def __init__(self,printtoggle):
      super().__init__()

      ### convolution layers
      self.conv1 = nn.Conv2d(4,32,kernel_size=3,dtype=torch.float32)
      self.conv1l = nn.Conv2d(1,1,kernel_size=3,dtype=torch.float32)

      self.conv2 = nn.Conv2d(32,63,kernel_size=4,dtype=torch.float32)
      self.conv2l = nn.Conv2d(1,1,kernel_size=4,dtype=torch.float32)

      self.conv3 = nn.Conv2d(64,64,kernel_size=3,dtype=torch.float32)
      # self.conv3l = nn.Conv2d(1,1,kernel_size=3,dtype=torch.float32)

      self.conv4 = nn.Conv2d(64,64,kernel_size=3,dtype=torch.float32)
      # self.conv4l = nn.Conv2d(1,1,kernel_size=3,dtype=torch.float32)

      self.fc1 = nn.Linear(9216,512)

      self.fc2 = nn.Linear(512,64)

      self.out = nn.Linear(64,1)
      self.print = printtoggle
    # forward pass
    def forward(self,x):
      x=x.float()
      layer=x[:,3,:,:] 
      print(f'Input: {x.shape}') if self.print else None

      
      x = F.relu(F.max_pool2d(self.conv1(x),2))
      layer=F.relu(F.max_pool2d(self.conv1l(layer),2))
      print(f'Layer conv1/pool1: {x.shape}') if self.print else None

      
      x = F.relu(F.max_pool2d(self.conv2(x),2))
      layer=F.relu(F.max_pool2d(self.conv1l(layer),2))
      print(f'Layer conv2/pool2: {x.shape}') if self.print else None
      y=torch.zeros((x.shape[0],64,x.shape[2],x.shape[3]))
      for i in range(63):
        y[:,i,:,:]=x[:,i,:,:]  

      y[:,63,:,:]=layer
      x=y
      x = F.relu(F.max_pool2d(self.conv3(x),2))

      x = F.relu(F.max_pool2d(self.conv4(x),2))

      nUnits = x.shape.numel()
      x=x.view(int(nUnits))
      if self.print: print(f'Vectorize: {x.shape}')
      
      x = F.relu(self.fc1(x))
      if self.print: print(f'Layer fc1: {x.shape}')
      x = F.relu(self.fc2(x))
      if self.print: print(f'Layer fc2: {x.shape}')
      x = self.out(x)
      if self.print: print(f'Layer out: {x.shape}')

      return x
  
  net = model(printtoggle)
  
  lossfun = nn.MSELoss()

  optimizer = torch.optim.Adam(net.parameters(),lr=.001)

  return net,lossfun,optimizer

In [97]:
net,lossfun,optimizer = createModel(True)

In [11]:
img=cv2.imread('./DS_Assignment_2/depthmap/1d5fee00-ada4-11eb-a80b-3f50af21830f.jpg')


In [12]:
img=cv2.resize(img,(224,224))

In [13]:
img,layer=imageread('./DS_Assignment_2/depthmap/1d5fee00-ada4-11eb-a80b-3f50af21830f.jpg',df.iloc[0]['Pose'])

In [15]:
def datapreprocess(path,pose):
    img,layer=imageread(path,pose)
    img=torch.tensor(img,dtype=torch.float16)
    img=img/255.0
    new_img=torch.zeros(img.shape[0],img.shape[1],4)
    # print(img.shape,' ',new_img.shape)
    new_img[:,:,0]=img[:,:,0]
    new_img[:,:,1]=img[:,:,1]
    new_img[:,:,2]=img[:,:,2]
    new_img[:,:,3]=layer
    new_img=torch.tensor(new_img/255.0,dtype=torch.float16)
    return new_img

In [16]:
img_tensor=datapreprocess('./DS_Assignment_2/depthmap/1d5fee00-ada4-11eb-a80b-3f50af21830f.jpg',df.iloc[0]['Pose'])

/var/folders/s2/b_s__vrj425fzynlpm1vql3w0000gn/T/ipykernel_39819/3928052728.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  new_img=torch.tensor(new_img/255.0,dtype=torch.float16)


In [17]:
yHat = net(img_tensor[None,:].permute(0, 3, 1, 2))

Input: torch.Size([1, 4, 224, 224])
Layer conv1/pool1: torch.Size([1, 32, 111, 111])
Layer conv2/pool2: torch.Size([1, 63, 54, 54])
Vectorize: torch.Size([9216])
Layer fc1: torch.Size([512])
Layer fc2: torch.Size([64])
Layer out: torch.Size([1])


In [99]:


def function2trainTheModel():
  numepochs = 10
  
  # create a new model
  net,lossfun,optimizer = createModel()

  losses= torch.zeros(numepochs)

  for epochi in range(numepochs):

    net.train()
    for X,z,y in zip(df['Depthmap Image'],df['Pose'],df['Height(cm)']):
      img=datapreprocess('./DS_Assignment_2/depthmap/'+X+'.jpg',z)
      # forward pass and loss
      yHat = net(img[None,:].permute(0, 3, 1, 2))
      loss = lossfun(yHat,torch.tensor([y]))

      # backprop
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      losses[epochi]=loss

    # test accuracy
    net.eval()

    with torch.no_grad(): # deactivates autograd
      yHat = net(img[None,:].permute(0, 3, 1, 2))

  return losses,net


In [100]:
losses,net = function2trainTheModel()

/var/folders/s2/b_s__vrj425fzynlpm1vql3w0000gn/T/ipykernel_39819/3928052728.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  new_img=torch.tensor(new_img/255.0,dtype=torch.float16)
